In [2]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('data/simple_two_bumps_3/flat_opt.msh')
driver=l.centralJoint()

mat = elastic_rods.RodMaterial('Rectangle', 2000, 0.3, [15.3,8.1], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('data/simple_two_bumps_3/design_parameters.txt'))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
# fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024, labelOffset=-0.5)
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [6]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
open_linkage(l, driver, np.deg2rad(15) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  0.432841654494593
0	3496.01	40.0702	40.0702	1	0
Backtracking failed.
0.477578951370361	3495.9390343551045	3495.9390343551045	3293.765758301431	69.5015788537617	132.67169719991142
target angle:  0.42935099599060433
0	3429.67	39.7882	39.7882	1	0
0.47376997863551196	3429.595644507125	3429.595644507125	3231.2570759470277	66.2133772397523	132.12519132034657
target angle:  0.42586033748661567
0	3363.86	39.5108	39.5108	1	0
0.46995799154841356	3363.788203063968	3363.788203063968	3169.1920062532968	63.03014901262319	131.56604779804834
target angle:  0.422369678982627
0	3298.59	39.2374	39.2374	1	0
0.4661430641763767	3298.518776126236	3298.518776126236	3107.574848837029	59.94868109262265	130.99524619658456
target angle:  0.41887902047863834
0	3233.86	38.9682	38.9682	1	0
0.46232525666852853	3233.7893719346034	3233.7893719346034	3046.409646075478	56.966202287182256	130.4135235719439
target angle:  0.4153883619746497
0	3169.67	38.7033	38.7033	1	0
0.45850461833279255	3169.6019551568197

In [ ]:
# Output fabrication data
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_meshID_1d1c1dc9-4638-474e-8f7f-6b95998b8a32.txt', zeroBasedIndexing=True)
np.savetxt('restlen_meshID_1d1c1dc9-4638-474e-8f7f-6b95998b8a32.txt',l.getPerSegmentRestLength())

In [ ]:
# Output rendering data (rest configuration)
writeRodSegments(l,'rod_segments_2bumps_rest.txt', zeroBasedIndexing=True)
np.savetxt('restlen_2bumps_rest.txt',l.getPerSegmentRestLength())
np.savetxt('normals_2bumps_rest.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_2bumps_rest.msh')

In [7]:
import compute_vibrational_modes
fixedVarsWithoutActuator = fixedVars[:-1]
# lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=fixedVarsWithoutActuator, mtype=compute_vibrational_modes.MassMatrixType.FULL)
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-2)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=5.0)
mview.show()

In [ ]:
# Output rendering data (deployed configuration)
writeRodSegments(l,'rod_segments_2bumps_95.txt', zeroBasedIndexing=True)
np.savetxt('restlen_2bumps_95.txt',l.getPerSegmentRestLength())
np.savetxt('normals_2bumps_95.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_2bumps_95.msh')